<a href="https://colab.research.google.com/github/ishika-b/neural-networks/blob/main/nnfl_assn2_q8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import scipy.io
from scipy.io import loadmat
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import  preprocessing
from sklearn.preprocessing import StandardScaler
normalizer = StandardScaler()
from sklearn.model_selection import train_test_split
import math

In [ ]:
np.random.seed(1)
lmat = scipy.io.loadmat('/content/drive/MyDrive/nnfl/nnfl2/data5.mat')
data = lmat['x']
df = pd.DataFrame(data)
data = np.array(df)
x = data [:,0:72]
y = data[:,72]

#change class  labels to -1 and 1 
for i in range(x.shape[0]):
  if y[i]==0:
    y[i]=-1
  
#normalize
##means = np.mean(x,axis = 0)
##column_wise_std_devs = np.std(x,axis = 0)
x = (x - np.mean(x,axis = 0))/ np.std(x,axis = 0)
  
#Split data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=1/8, random_state=1)

m,n = x_train.shape
m_test, n_test = x_test.shape

mus = np.zeros(m) #mus initialised to 0 
b = 0 #biases initialised to 0
E = np.zeros(m) 
eta = 0
H = 0
L = 0
K = x_train@x_train.T
iters = 0
max_iters = 100
bound = 1e-2
C = 13
while (iters < max_iters):
  count_change = 0
  #print("iteration",iters)
  for i in range(m):
    f = b + np.sum(np.multiply(np.multiply(mus,y_train),K[:,i]))
    E[i] = f - y_train[i]
    if ((y_train[i]*E[i]<-bound and mus[i]<C) or (y_train[i]*E[i]>bound and mus[i]>0)):
      j = math.ceil((m-1)*np.random.rand())
      while j==i:
        j = math.ceil((m-1)*np.random.rand())
      E[j] = b + np.sum(np.multiply(np.multiply(mus,y_train),K[:,j])) - y_train[j]
      mus_i_old = mus[i]
      mus_j_old = mus[j]

      if y_train[i]==y_train[j]:
        L = max(0.0,(mus[j]+mus[i]-C))
        H = min(C, (mus[j]+mus[i]))
      else:
        L = max(0, mus[j]-mus[i])
        H = min(C, C+mus[j]-mus[i])

      if L==H:
        continue
      eta = 2*K[i][j]-K[i][i]-K[j][j]
      if eta>=0:
        continue
      mus[j] = mus[j] - (y_train[j]*(E[i]-E[j]))/eta
      #print(j,mus[j])
      #print(L,H)
      mus[j] = min(H,mus[j])
      mus[j] = max(L,mus[j])
      #print(j,mus[j])
      
      if (abs(mus[j]-mus_j_old)<bound):
        mus[j] = mus_j_old
        continue
      #print(abs(mus[j]-mus_j_old))
      mus[i] = mus[i] + y_train[i]*y_train[j]*(mus_j_old-mus[j])
      b1 = b -E[i]-y_train[i]*(mus[i]-mus_i_old)*K[i][i]-y_train[j]*(mus[j]-mus_j_old)*K[i][j]
      b2 = b -E[j]-y_train[i]*(mus[i]-mus_i_old)*K[i][j]-y_train[j]*(mus[j]-mus_j_old)*K[j][j]
      if (0 < mus[i] and mus[i] < C):
        b = b1
      elif (0 < mus[j] and mus[j] < C):
        b = b2
      else:
        b = (b1+b2)/2
      count_change = count_change + 1
      #print("count-change",count_change)
    if (count_change == 0):
      iters = iters + 1
      #print(mus)
    else:
      iters = 0

  w = 0
  num_sv = 0
  for i in range(m):
    if mus[i]>0: #only these are the support vectors 
      w += mus[i]*y_train[i]*x[i,:]
      num_sv += 1

0.8488372093023255


In [ ]:
m_val, n_val = x_val.shape
y_pred_val = np.zeros(m_val)
for i in range(m_val):
  sum = 0
  for j in range(m):
    if mus[j]>=0: #only these are the support vectors 
      sum += mus[j]*y_train[j]*np.dot(x_train[j],x_val[i].T)
  sum += b
  y_pred_val[i] = np.sign(sum)

acc = 0
for i in range(m_val):
  if (y_pred_val[i] == -1 and y_val[i] == -1) or (y_pred_val[i] == 1 and y_val[i] == 1):
    acc += 1
acc = acc/m_val
print(acc)

0.8558139534883721


In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, y_pred_val)
print("validation class 0 accuracy:", cm[0,0]/(cm[0,0]+cm[0,1]))
print("validation class 1 accuracy :", cm[1,1]/(cm[1,1]+cm[1,0]))

validation class 0 accuracy: 0.8679245283018868
validation class 1 accuracy : 0.8440366972477065


In [ ]:
y_pred = np.zeros(m_test)
for i in range(m_test):
  sum = 0
  for j in range(m):
    if mus[j]>=0: #only these are the support vectors 
      sum += mus[j]*y_train[j]*np.dot(x_train[j],x_test[i].T)
  sum += b
  y_pred[i] = np.sign(sum)


acc = 0
for i in range(m_test):
  if (y_pred[i] == -1 and y_test[i] == -1) or (y_pred[i] == 1 and y_test[i] == 1):
    acc += 1
acc = acc/m_test
print("overall testing acuracy:",acc)

overall testing acuracy: 0.8488372093023255


In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, y_pred_val)
print("validation class 0 accuracy:", cm[0,0]/(cm[0,0]+cm[0,1]))
print("validation class 1 accuracy :", cm[1,1]/(cm[1,1]+cm[1,0]))

validation class 0 accuracy: 0.8679245283018868
validation class 1 accuracy : 0.8440366972477065
